In [ ]:
!pip install PyQt5 opencv-python numpy matplotlib torch torchvision trimesh open3d scikit-image scipy

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, 
                             QLabel, QPushButton, QComboBox, QFileDialog, QMessageBox, 
                             QGroupBox, QProgressBar, QCheckBox, QSpinBox, QDoubleSpinBox,
                             QTabWidget, QSizePolicy, QScrollArea, QLineEdit)
from PyQt5.QtCore import Qt, QThread, pyqtSignal
from PyQt5.QtGui import QPixmap, QImage, QIcon
import cv2
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from torchvision.transforms import functional as F
import trimesh
from skimage import measure
import open3d as o3d
import torch
import torch.hub
from torchvision import transforms
import time


In [ ]:
# -------------------- Original Functions --------------------

def detect_objects_with_yolo(image):
    """Detect objects in the image using YOLO"""
    print("Loading YOLO model...")
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    
    results = model(image)
    detections = results.xyxy[0].cpu().numpy()
    class_names = [model.names[int(cls)] for cls in detections[:, 5]]
    
    return detections, class_names

def yolo_based_roi_selection(image, min_confidence=0.4):
    """Select region of interest based on YOLO object detection"""
    detections, class_names = detect_objects_with_yolo(image)
    
    if len(detections) > 0:
        valid_detections = detections[detections[:, 4] >= min_confidence]
        
        if len(valid_detections) > 0:
            best_idx = np.argmax([d[4] for d in valid_detections])
            selected_det = valid_detections[best_idx]
            class_name = class_names[best_idx]
            
            x1, y1, x2, y2 = map(int, selected_det[:4])
            padding_percent = 0.1
            pad_x = int((x2 - x1) * padding_percent)
            pad_y = int((y2 - y1) * padding_percent)
            
            h, w = image.shape[:2]
            x1 = max(0, x1 - pad_x)
            y1 = max(0, y1 - pad_y)
            x2 = min(w, x2 + pad_x)
            y2 = min(h, y2 + pad_y)
            
            return (x1, y1, x2, y2), class_name
    
    return enhanced_auto_select_roi(image), "unknown"

def auto_select_depth_model(image):
    """Select best depth estimation model based on image characteristics"""
    img_size = image.shape[0] * image.shape[1]
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    
    if img_size > 1_000_000 and laplacian_var > 100:
        model_type = "DPT_Large"
    elif laplacian_var > 70:
        model_type = "DPT_Hybrid"
    else:
        model_type = "MiDaS_small"
    
    midas = torch.hub.load("intel-isl/MiDaS", model_type)
    midas.to('cuda' if torch.cuda.is_available() else 'cpu').eval()
    
    if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
        transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
    else:
        transform = torch.hub.load("intel-isl/MiDaS", "transforms").small_transform
        
    return midas, transform, model_type

def enhanced_auto_select_roi(image):
    """Automatic selection of ROI using multiple techniques"""
    height, width = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh_adaptive = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY_INV, 11, 2)
    _, thresh_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    combined_mask = cv2.bitwise_or(thresh_adaptive, thresh_otsu)
    
    kernel = np.ones((5, 5), np.uint8)
    mask_eroded = cv2.erode(cv2.dilate(combined_mask, kernel, iterations=1), kernel, iterations=1)
    
    contours, _ = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c for c in contours if cv2.contourArea(c) > max(500, (width * height * 0.01))]
    
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        padding_percent = 0.15
        padding_x = int(w * padding_percent)
        padding_y = int(h * padding_percent)
        
        x1 = max(0, x - padding_x)
        y1 = max(0, y - padding_y)
        x2 = min(width, x + w + padding_x)
        y2 = min(height, y + h + padding_y)
        
        if (x2 - x1) < 50 or (y2 - y1) < 50:
            center_x, center_y = width // 2, height // 2
            min_dim = max(100, min(width, height) // 4)
            x1 = max(0, center_x - min_dim // 2)
            y1 = max(0, center_y - min_dim // 2)
            x2 = min(width, center_x + min_dim // 2)
            y2 = min(height, center_y + min_dim // 2)
        
        return (x1, y1, x2, y2)
    
    center_x, center_y = width // 2, height // 2
    size = min(width, height) // 2
    return max(0, center_x - size), max(0, center_y - size), min(width, center_x + size), min(height, center_y + size)

def get_object_specific_params(class_name):
    """Get parameters for different object types"""
    object_params = {
        'person': {'scale_z': 0.6, 'smoothing': 5, 'mesh_detail': 9, 'outlier_removal': 2.0},
        'car': {'scale_z': 0.5, 'smoothing': 3, 'mesh_detail': 9, 'outlier_removal': 1.8},
        'bottle': {'scale_z': 1.0, 'smoothing': 7, 'mesh_detail': 8, 'outlier_removal': 2.2},
        'default': {'scale_z': 0.5, 'smoothing': 5, 'mesh_detail': 8, 'outlier_removal': 2.0}
    }
    return object_params.get(class_name.lower(), object_params['default'])

def get_enhanced_depth(image, midas, transform):
    """Get depth with additional post-processing"""
    input_batch = transform(image).to('cuda' if torch.cuda.is_available() else 'cpu')
    
    with torch.no_grad():
        prediction = midas(input_batch)
    
    depth = prediction.squeeze().cpu().numpy()
    depth_normalized = cv2.normalize(depth, None, 0, 1, cv2.NORM_MINMAX)
    depth_normalized = (depth_normalized * 255).astype(np.uint8)
    depth_filtered = cv2.bilateralFilter(depth_normalized, 9, 75, 75)
    depth_filtered = depth_filtered.astype(np.float32) / 255
    depth_filtered = depth_filtered * (depth.max() - depth.min()) + depth.min()
    
    percentile_low, percentile_high = np.percentile(depth_filtered, [2, 98])
    depth_filtered = np.clip(depth_filtered, percentile_low, percentile_high)
    
    return depth_filtered

def extract_3d_from_roi(depth_map, roi, image, object_params):
    """Extract 3D points with object-specific parameters"""
    x1, y1, x2, y2 = roi
    roi_depth = depth_map[y1:y2, x1:x2]
    roi_image = image[y1:y2, x1:x2]
    
    roi_depth = cv2.GaussianBlur(roi_depth, (3, 3), 0)
    h, w = roi_depth.shape
    yy, xx = np.mgrid[0:h, 0:w]
    
    depth_min = np.percentile(roi_depth, 5)
    depth_max = np.percentile(roi_depth, 95)
    depth_scaled = np.clip(roi_depth, depth_min, depth_max)
    depth_scaled = (depth_scaled - depth_min) / (depth_max - depth_min)
    
    mask = ~np.isnan(depth_scaled)
    xx_valid = xx[mask]
    yy_valid = yy[mask]
    depth_valid = depth_scaled[mask]
    
    num_points = np.sum(mask)
    points = np.zeros((num_points, 6))
    
    scale_x = 1.0
    scale_y = h / w if h > w else 1.0
    scale_z = object_params['scale_z']
    
    points[:, 0] = xx_valid * scale_x
    points[:, 1] = yy_valid * scale_y
    points[:, 2] = depth_valid * scale_z
    
    for i, (x, y) in enumerate(zip(xx_valid, yy_valid)):
        points[i, 3:6] = roi_image[y, x] / 255.0
    
    points[:, 0] = (points[:, 0] / w - 0.5) * 2
    points[:, 1] = (points[:, 1] / h - 0.5) * 2
    
    return points

def generate_enhanced_3d_model(points, method="mesh", object_params=None):
    """Generate 3D model with object-specific parameters"""
    if object_params is None:
        object_params = get_object_specific_params("default")
        
    if method == "pointcloud":
        return points
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    pcd.colors = o3d.utility.Vector3dVector(points[:, 3:6])
    
    if len(pcd.points) > 50000:
        pcd = pcd.voxel_down_sample(voxel_size=0.01)
    
    pcd, _ = pcd.remove_statistical_outlier(
        nb_neighbors=20, 
        std_ratio=object_params['outlier_removal']
    )
    
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=15)
    
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, 
        depth=object_params['mesh_detail'],
        width=0, 
        scale=1.1, 
        linear_fit=False
    )
    
    density_threshold = np.quantile(densities, 0.1)
    vertices_to_remove = densities < density_threshold
    mesh.remove_vertices_by_mask(vertices_to_remove)
    
    if len(mesh.triangles) > 100000:
        mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=100000)
    
    mesh = mesh.filter_smooth_taubin(number_of_iterations=object_params['smoothing'])
    
    vertices = np.asarray(mesh.vertices)
    vertex_colors = np.zeros((len(vertices), 3))
    
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    for i, vertex in enumerate(vertices):
        _, idx, _ = pcd_tree.search_knn_vector_3d(vertex, 1)
        if idx:
            vertex_colors[i] = np.asarray(pcd.colors)[idx[0]]
    
    mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
    
    tri_mesh = trimesh.Trimesh(vertices=vertices,
                              faces=np.asarray(mesh.triangles),
                              vertex_colors=vertex_colors)
    
    tri_mesh.remove_duplicate_faces()
    tri_mesh.remove_degenerate_faces()
    tri_mesh.remove_unreferenced_vertices()
    
    return tri_mesh

def save_3d_model(model, filename, method="mesh"):
    """Save 3D model in appropriate format"""
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    if method == "pointcloud":
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(model[:, :3])
        pcd.colors = o3d.utility.Vector3dVector(model[:, 3:6])
        o3d.io.write_point_cloud(filename, pcd)
    elif method == "mesh":
        ext = os.path.splitext(filename)[1].lower()
        if ext == '.ply':
            model.export(filename, file_type='ply')
        elif ext == '.obj':
            model.export(filename, file_type='obj')
        elif ext == '.stl':
            model.export(filename, file_type='stl')
        elif ext == '.glb':
            model.export(filename, file_type='glb')
        else:
            filename = os.path.splitext(filename)[0] + '.ply'
            model.export(filename, file_type='ply')

# -------------------- Worker Thread --------------------

class WorkerThread(QThread):
    progress_signal = pyqtSignal(int, str)
    finished_signal = pyqtSignal(object, str, str, np.ndarray, np.ndarray, tuple)
    error_signal = pyqtSignal(str)
    preview_signal = pyqtSignal(np.ndarray, np.ndarray, tuple)

    def __init__(self, image_path, roi_method, output_method, output_dir, params):
        super().__init__()
        self.image_path = image_path
        self.roi_method = roi_method
        self.output_method = output_method
        self.output_dir = output_dir
        self.params = params

    def run(self):
        try:
            # Load image
            self.progress_signal.emit(5, "Loading image...")
            image = cv2.cvtColor(cv2.imread(self.image_path), cv2.COLOR_BGR2RGB)
            
            # Select ROI
            self.progress_signal.emit(15, "Selecting region of interest...")
            if self.roi_method == "auto":
                roi, class_name = yolo_based_roi_selection(image)
            elif self.roi_method == "manual":
                roi = enhanced_auto_select_roi(image)
                class_name = "unknown"
            else:
                roi = enhanced_auto_select_roi(image)
                class_name = "unknown"
            
            # Get parameters
            self.progress_signal.emit(25, "Configuring parameters...")
            object_params = {
                'scale_z': self.params['scale_z'],
                'smoothing': self.params['smoothing'],
                'mesh_detail': self.params['detail'],
                'outlier_removal': self.params['outlier']
            }
            
            # Initialize model
            self.progress_signal.emit(35, "Initializing depth model...")
            midas, transform, model_type = auto_select_depth_model(image)
            
            # Generate depth map
            self.progress_signal.emit(50, "Generating depth map...")
            depth = get_enhanced_depth(image, midas, transform)
            
            # Extract 3D points
            self.progress_signal.emit(65, "Extracting 3D points...")
            points_3d = extract_3d_from_roi(depth, roi, image, object_params)
            
            # Generate 3D model
            self.progress_signal.emit(80, "Generating 3D model...")
            model_3d = generate_enhanced_3d_model(points_3d, self.output_method, object_params)
            
            # Emit preview data
            self.preview_signal.emit(image, depth, roi)
            
            # Save model
            self.progress_signal.emit(90, "Saving results...")
            base_name = os.path.splitext(os.path.basename(self.image_path))[0]
            output_file = os.path.join(self.output_dir, f"{base_name}_{self.output_method}_{class_name}")
            
            if self.output_method == "pointcloud":
                output_file += ".ply"
            else:
                output_file += ".obj"
            
            save_3d_model(model_3d, output_file, method=self.output_method)
            
            self.finished_signal.emit(model_3d, class_name, model_type, image, depth, roi)
            self.progress_signal.emit(100, "Finished!")
            
        except Exception as e:
            self.error_signal.emit(str(e))

# -------------------- Main Window --------------------

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Model Generator")
        self.setGeometry(100, 100, 1200, 800)
        
        # Central widget and layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)
        
        # Left panel (controls)
        left_panel = QWidget()
        left_panel.setMaximumWidth(300)
        left_layout = QVBoxLayout(left_panel)
        
        # Right panel (display)
        right_panel = QTabWidget()
        right_panel.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        
        # Add panels to main layout
        main_layout.addWidget(left_panel)
        main_layout.addWidget(right_panel)
        
        # -------------------- Left Panel Controls --------------------
        
        # Input group
        input_group = QGroupBox("Input Settings")
        input_layout = QVBoxLayout()
        
        self.image_path_edit = QLineEdit()
        self.image_path_edit.setPlaceholderText("Select image file...")
        browse_btn = QPushButton("Browse...")
        browse_btn.clicked.connect(self.browse_image)
        
        input_layout.addWidget(QLabel("Image File:"))
        input_layout.addWidget(self.image_path_edit)
        input_layout.addWidget(browse_btn)
        input_group.setLayout(input_layout)
        
        # ROI group
        roi_group = QGroupBox("ROI Selection")
        roi_layout = QVBoxLayout()
        
        self.roi_method_combo = QComboBox()
        self.roi_method_combo.addItems(["Auto (YOLO)", "Auto (Traditional)", "Manual"])
        
        roi_layout.addWidget(QLabel("Selection Method:"))
        roi_layout.addWidget(self.roi_method_combo)
        roi_group.setLayout(roi_layout)
        
        # Output group
        output_group = QGroupBox("Output Settings")
        output_layout = QVBoxLayout()
        
        self.output_method_combo = QComboBox()
        self.output_method_combo.addItems(["Mesh", "Point Cloud"])
        
        self.output_dir_edit = QLineEdit()
        self.output_dir_edit.setPlaceholderText("Select output directory...")
        output_browse_btn = QPushButton("Browse...")
        output_browse_btn.clicked.connect(self.browse_output_dir)
        
        output_layout.addWidget(QLabel("Output Type:"))
        output_layout.addWidget(self.output_method_combo)
        output_layout.addWidget(QLabel("Output Directory:"))
        output_layout.addWidget(self.output_dir_edit)
        output_layout.addWidget(output_browse_btn)
        output_group.setLayout(output_layout)
        
        # Parameters group
        params_group = QGroupBox("3D Generation Parameters")
        params_layout = QVBoxLayout()
        
        self.scale_z_spin = QDoubleSpinBox()
        self.scale_z_spin.setRange(0.1, 2.0)
        self.scale_z_spin.setValue(0.5)
        self.scale_z_spin.setSingleStep(0.1)
        
        self.smoothing_spin = QSpinBox()
        self.smoothing_spin.setRange(1, 10)
        self.smoothing_spin.setValue(5)
        
        self.detail_spin = QSpinBox()
        self.detail_spin.setRange(5, 12)
        self.detail_spin.setValue(8)
        
        self.outlier_spin = QDoubleSpinBox()
        self.outlier_spin.setRange(1.0, 3.0)
        self.outlier_spin.setValue(2.0)
        self.outlier_spin.setSingleStep(0.1)
        
        params_layout.addWidget(QLabel("Depth Scale (Z):"))
        params_layout.addWidget(self.scale_z_spin)
        params_layout.addWidget(QLabel("Smoothing Iterations:"))
        params_layout.addWidget(self.smoothing_spin)
        params_layout.addWidget(QLabel("Mesh Detail Level:"))
        params_layout.addWidget(self.detail_spin)
        params_layout.addWidget(QLabel("Outlier Removal:"))
        params_layout.addWidget(self.outlier_spin)
        params_group.setLayout(params_layout)
        
        # Process controls
        process_group = QGroupBox("Processing")
        process_layout = QVBoxLayout()
        
        self.progress_bar = QProgressBar()
        self.progress_label = QLabel("Ready")
        self.process_btn = QPushButton("Generate 3D Model")
        self.process_btn.clicked.connect(self.start_processing)
        self.cancel_btn = QPushButton("Cancel")
        self.cancel_btn.setEnabled(False)
        
        process_layout.addWidget(self.progress_bar)
        process_layout.addWidget(self.progress_label)
        process_layout.addWidget(self.process_btn)
        process_layout.addWidget(self.cancel_btn)
        process_group.setLayout(process_layout)
        
        # Add all groups to left panel
        left_layout.addWidget(input_group)
        left_layout.addWidget(roi_group)
        left_layout.addWidget(output_group)
        left_layout.addWidget(params_group)
        left_layout.addWidget(process_group)
        left_layout.addStretch()
        
        # -------------------- Right Panel Tabs --------------------
        
        # Input image tab
        self.input_tab = QWidget()
        self.input_layout = QVBoxLayout(self.input_tab)
        
        self.input_figure = Figure()
        self.input_canvas = FigureCanvas(self.input_figure)
        self.input_toolbar = NavigationToolbar(self.input_canvas, self)
        
        self.input_layout.addWidget(self.input_toolbar)
        self.input_layout.addWidget(self.input_canvas)
        
        # Depth map tab
        self.depth_tab = QWidget()
        self.depth_layout = QVBoxLayout(self.depth_tab)
        
        self.depth_figure = Figure()
        self.depth_canvas = FigureCanvas(self.depth_figure)
        self.depth_toolbar = NavigationToolbar(self.depth_canvas, self)
        
        self.depth_layout.addWidget(self.depth_toolbar)
        self.depth_layout.addWidget(self.depth_canvas)
        
        # Results tab
        self.results_tab = QWidget()
        self.results_layout = QVBoxLayout(self.results_tab)
        
        self.results_label = QLabel("Results will be shown here after processing")
        self.results_label.setAlignment(Qt.AlignCenter)
        self.results_layout.addWidget(self.results_label)
        
        # Add tabs to right panel
        right_panel.addTab(self.input_tab, "Input Image")
        right_panel.addTab(self.depth_tab, "Depth Map")
        right_panel.addTab(self.results_tab, "Results")
        
        # Worker thread
        self.worker_thread = None
        
        # Initialize CUDA status
        self.update_cuda_status()
        
    def update_cuda_status(self):
        cuda_available = torch.cuda.is_available()
        status = "CUDA: " + ("Available" if cuda_available else "Not Available")
        self.statusBar().showMessage(status)
        
    def browse_image(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Select Image", "", 
            "Image Files (*.png *.jpg *.jpeg *.bmp)"
        )
        if file_path:
            self.image_path_edit.setText(file_path)
            self.display_input_image(file_path)
            
    def browse_output_dir(self):
        dir_path = QFileDialog.getExistingDirectory(self, "Select Output Directory")
        if dir_path:
            self.output_dir_edit.setText(dir_path)
            
    def display_input_image(self, image_path):
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        self.input_figure.clear()
        ax = self.input_figure.add_subplot(111)
        ax.imshow(image)
        ax.axis('off')
        self.input_canvas.draw()
        
    def start_processing(self):
        if not self.image_path_edit.text():
            QMessageBox.warning(self, "Error", "Please select an image file first!")
            return
            
        if not self.output_dir_edit.text():
            QMessageBox.warning(self, "Error", "Please select an output directory!")
            return
            
        # Get parameters from UI
        roi_method = ["auto", "auto", "manual"][self.roi_method_combo.currentIndex()]
        output_method = ["mesh", "pointcloud"][self.output_method_combo.currentIndex()]
        
        params = {
            'scale_z': self.scale_z_spin.value(),
            'smoothing': self.smoothing_spin.value(),
            'detail': self.detail_spin.value(),
            'outlier': self.outlier_spin.value()
        }
        
        # Create and start worker thread
        self.worker_thread = WorkerThread(
            self.image_path_edit.text(),
            roi_method,
            output_method,
            self.output_dir_edit.text(),
            params
        )
        
        # Connect signals
        self.worker_thread.progress_signal.connect(self.update_progress)
        self.worker_thread.finished_signal.connect(self.processing_finished)
        self.worker_thread.error_signal.connect(self.processing_error)
        self.worker_thread.preview_signal.connect(self.update_previews)
        
        # Update UI
        self.process_btn.setEnabled(False)
        self.cancel_btn.setEnabled(True)
        self.progress_bar.setValue(0)
        self.progress_label.setText("Processing...")
        
        # Start thread
        self.worker_thread.start()
        
    def update_progress(self, value, message):
        self.progress_bar.setValue(value)
        self.progress_label.setText(message)
        
    def update_previews(self, image, depth, roi):
        # Update input image with ROI
        self.input_figure.clear()
        ax = self.input_figure.add_subplot(111)
        ax.imshow(image)
        x1, y1, x2, y2 = roi
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.axis('off')
        self.input_canvas.draw()
        
        # Update depth map
        self.depth_figure.clear()
        ax = self.depth_figure.add_subplot(111)
        ax.imshow(depth, cmap='plasma')
        ax.axis('off')
        self.depth_canvas.draw()
        
    def processing_finished(self, model_3d, class_name, model_type, image, depth, roi):
        self.process_btn.setEnabled(True)
        self.cancel_btn.setEnabled(False)
        self.progress_label.setText("Finished!")
        
        # Update results tab
        self.results_label.setText(
            f"3D Model Generation Complete!\n\n"
            f"Object Type: {class_name}\n"
            f"Depth Model: {model_type}\n"
            f"ROI: {roi}\n\n"
            f"Model saved to output directory."
        )
        
        QMessageBox.information(self, "Success", "3D model generation completed successfully!")
        
    def processing_error(self, error_message):
        self.process_btn.setEnabled(True)
        self.cancel_btn.setEnabled(False)
        self.progress_label.setText("Error occurred")
        
        QMessageBox.critical(self, "Error", f"An error occurred:\n{error_message}")
        
    def closeEvent(self, event):
        if self.worker_thread and self.worker_thread.isRunning():
            self.worker_thread.terminate()
            self.worker_thread.wait()
        event.accept()

# -------------------- Application --------------------

if __name__ == "__main__":
    app = QApplication(sys.argv)
    
    # Set application style
    app.setStyle('Fusion')
    
    # Create and show main window
    window = MainWindow()
    window.show()
    
    sys.exit(app.exec_())